In [ ]:
! pip install transformers 
! pip install datasets 
! pip install huggingface_hub
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
datasets=load_dataset("squad")
datasets

In [ ]:
import copy
dstemp = copy.deepcopy(datasets)

In [ ]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")


In [ ]:
model=model.to('cuda')

In [ ]:

c=0
e=0
t=0
Telugutranslated = 0
Bengtranslated = 0
for idx,i in enumerate(datasets['train']):
    #---------English to Bengali
    model_inputs = tokenizer(i['question'], return_tensors="pt")
    model_inputs=model_inputs.to('cuda')
    # translate from English to Bengali
    Beng_generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id["bn_IN"]
    )
    Bengtranslated = tokenizer.batch_decode(Beng_generated_tokens, skip_special_tokens=True)
    #--------English to Telugu
    model_inputs = tokenizer(i['question'], return_tensors="pt")
    model_inputs=model_inputs.to('cuda')
    # translate from English to Bengali
    Beng_generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id["te_IN"]
    )
    Telugutranslated = tokenizer.batch_decode(Beng_generated_tokens, skip_special_tokens=True)
    add_it = {
        'id': datasets['train'][idx]['id'],
        'title': datasets['train'][idx]['title'],
        'context': datasets['train'][idx]['context'],
        'question': Bengtranslated[0],
        'answers': copy.deepcopy(datasets['train'][idx]['answers'])
    }
    dstemp['train'] = dstemp['train'].add_item(copy.deepcopy(add_it))
    add_it['question'] = Telugutranslated[0]
    dstemp['train'] = dstemp['train'].add_item(copy.deepcopy(add_it))
    t+=1
    c+=1
    if c%10==0:
        print(c,t,e)
    if c==10000:
        break
    
dstemp

In [ ]:
#from huggingface_hub import notebook_login
#notebook_login()

In [ ]:
#dstemp.push_to_hub("Gautam9595/Squad_Translated")

In [ ]:
datasets2=load_dataset("Gautam9595/Squad_Translated")
datasets2